---

# Livro "Candlestick - Um método para ampliar lucros na Bolsa de Valores"

---

Com o objetivo de aprender sobre os Padrões de Candlestick, este notebook operacionaliza um por um, identificando cada candlestick e gerando um esboço de função para usos futuros. 

Nos textos a seguir, você encontrará os trechos que julguei mais importante no livro, junto com a respectiva implementação.

## 1. Configurando o Notebook

### 1.1. Obtendo dados

In [ ]:
from backtesting import Cruzamentos
from conexao import importar_dados_historicos
import vectorbt as vbt 


import pandas as pd 
import numpy as np 
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import pytz 

import MetaTrader5 as mt5
import ta
import plotly.graph_objects as go
import mplfinance as mpf

mt5.initialize()

timezone = pytz.timezone("America/Sao_Paulo")  
tickers = ['WDO$', 'PETR4', 'VALE3', 'ITSA4', 'WIN$', 'ABEV3']


for ticker in tickers:
    selected = mt5.symbol_select(ticker, True)
    if not selected:
        print(ticker, "não localizado")


start = datetime(2022, 1, 1)
end = datetime.today()
timeframe = mt5.TIMEFRAME_D1
dados_gerais = dict()


for ticker in tickers:
    dados_brutos = importar_dados_historicos(ticker, start, end, timeframe)
    dados_gerais[ticker] = dados_brutos 

dados = dados_gerais[tickers[0]].copy()

### 1.2. Criando função para mostrar gráficos dos padrões identificados

In [ ]:
def plotar_candle(df, data_candle, diario=True, qtde_ticks = 6):
    
    df_figure = df.loc[data_candle-relativedelta(days=qtde_ticks):data_candle+relativedelta(days=qtde_ticks)]
    

    fig_topos_fundos = go.Figure(data=[go.Candlestick(
                                            name='',
                                            x=df_figure.index,
                                            open=df_figure.open,
                                            close=df_figure.close, 
                                            low=df_figure.low,
                                            high=df_figure.high, 
                                            increasing_line_color='green',
                                            decreasing_line_color='red',
                                            showlegend=False
                                )])
    
    fig_topos_fundos.add_vrect(
        x0=data_candle - pd.Timedelta(hours=12), 
        x1=data_candle + pd.Timedelta(hours=12),
        fillcolor="gray", opacity=0.25, line_width=0
    )


    if diario:
        fig_topos_fundos.update_xaxes(rangebreaks=[dict(bounds=['sat', 'mon'])])

    fig_topos_fundos.update_layout(
                                xaxis_rangeslider_visible=False,
                                title_text=f'<b>Padrão de Candlestick - {df_figure.ticker.iloc[0]} </b>',
                                template='none',
                                margin=dict(l=75, r=75, t=75, b=75),
                                width=800,
                                height=600
                            )
    fig_topos_fundos.show()

## 2. Padrões ambivalentes

"Esses padrões têm particularidades que fazem com que tenham significado diferente mesmo dentro de uma mesma tendência, dependendo do ponto em que surgem na sua trajetória."

### 2.1. Marobozu

"[...] nada mais é que um candle com corpo muito longo, com nenhuma sombra ou com sombras bem pequenas, se presentes."

"Para eleger um candle como Marobozu, ele deve destoar em tamanho dos demais candles presentes na figura gráfica analisada".

"A ausência de sombras nas duas extremidades da figura também deve ser considerada como fator de relevância na análise",


"A frequência com que aparece numa figura gráfica um pouco mais ampla também serve como indicador de relevância: quanto mais rara a aparição dele na figura gráfica total, mais significado tem na análise".


"Normalmente um Marobozu só é indicador de força no sentido da tendência quando aparece no início dela, nos primeiros pregões. Localizado [...] dez ou doze pregões depois do estabelecimento de uma tendência, um Marobozu a favor dela pode representar, em vez de força, uma exaustão".

"Uma interpretação importante do Marobozu é a que se faz quando ele aparece em sentido contrário à tendência [...]. A primeira análise é verificar se o Marobozu compõe, com outros candles, alguma figura reversida de maior relevância. Se isso não ocorrer, atentar para aspectos fundamentalistas do papel, como uma redução significativa no critério de distribuição de dividendos [...] ou algum fato relevante capaz de afetar a situação econômica do setor de atividade ou a própria empresa, de maneira a diminuir o interesse do mercado pelo papel."

In [ ]:
def encontrar_marobozus(dados, sombra_max=0.01):

    # Buscando candles com sombras de até 1% do corpo 
    sombra_max = 0.01
    corpo = abs(dados.close - dados.open)
    # Independentemente de ser candle de alta ou baixa
    sombra_sup = dados.high - dados[['close', 'open']].max(axis=1)
    sombra_inf = dados[['close', 'open']].min(axis=1) - dados.low

    sinal_candle = (sombra_sup <= sombra_max * corpo) & (sombra_inf <= sombra_max * corpo) 

    return sinal_candle

In [ ]:
padrao_candle = {}

for ticker, df_ohlc in dados_gerais.items():
    sinal_candle = encontrar_marobozus(df_ohlc)
    if sinal_candle.sum() > 0:
        padrao_candle[ticker] = sinal_candle[sinal_candle]


In [ ]:
plotar_candle(dados_gerais['VALE3'], data_candle = padrao_candle['VALE3'].index[0], qtde_ticks = 10)

### 2.2. Doji

"O que caracteriza um Doji é o fato de os preços de abertura e de fechamento serem idênticos. [...] O resultado é um candle sem corpo definido"

"Dadas as suas características, o Doji pode apresentar várias configurações com nomes distintos [...]:

- Doji Libélula (Dragonfly Doji): Quando os preços de abertura e fechamento coincidem também com a máxima do dia, formando uma figura em formato de T.
- Doji Lápide (Gravestone Doji): Quando os preços de abertura e fechamento coincidem também com a mínima do dia.
- Doji Pernalta (Long-Legged Doji): Quando a sombra inferior é muito maior que a superior.
- Doji Quatro-Preços (Four-Price Doji): Quando os preços de abertura e fechamento coincidem também com a máxima e a mínima do dia, [...]. Este Doji é muito comum em papéis de baixa liquidez, quando ocorre somente um negócio no dia"

"Para caracterizar um Doji, a figura não depende de seu posicionamento, nem da direção da tendência [...]. Em qualquer momento gráfico, será sempre um Doji e terá um mesmo significado: um mercado indeciso."

"[...] a identificação de um mercado indeciso só tem relevância durante o desenvolvimento de tendências. Dessa forma, um Doji que apareça em um mercado lateral (acumulação) ou mesmo numa faixa de negociação estreita não terá grande significado, [...].

"De forma genérica, o Doji representa sempre um sentimento de indecisão no mercado, podendo figurar uma possível reversão [...]. Essa indecisão é tão mais significativa quanto menores forem as sombras do Doji e quanto maiores forem os candles que o antecedem. Por essa razão, ele é visto como um sinal de esgotamento da tendência e de equilíbrio entre as forças de compra e venda."

"Um Doji é mais relevante em um topo do que em um fundo [...]"

"Devemos dar mais atenção ao Doji quando ele surgir em mercados extremamente sobrecomprados ou sobrevendidos [...] principalmente se houver, concomitantemente, outros indicadores de reversão [...]"

In [ ]:
def encontrar_doji(dados, corpo_max=0.01):

    # Buscando candles com corpo de até "corpo_max" 
    corpo = abs(dados.close - dados.open)
    
    sinal_candle = corpo <= corpo_max

    return sinal_candle

In [ ]:
padrao_candle = {}

for ticker, df_ohlc in dados_gerais.items():
    sinal_candle = encontrar_doji(df_ohlc)
    if sinal_candle.sum() > 0:
        padrao_candle[ticker] = sinal_candle[sinal_candle]
padrao_candle

In [ ]:
plotar_candle(dados_gerais['WDO$'], data_candle = padrao_candle['WDO$'].index[0], qtde_ticks = 20)

### 2.3. Spinning Top

"[...] candle de pequenas proporções, com um corpo pequeno e duas sombras idênticas, também pequenas [...]"

"Esse <b>não</b> é um padrão de grande relevância: só tem significado prático quando destoar dos demais candles (anteriores e posteriores) [...]"

"Assim como o Doji, o Spinning Top representa indecisão no mercado, todavia não com o mesmo grau de intensidade."

In [ ]:
def encontrar_spinning_top(dados, qtde_candles_proximos = 5, percentual_diff_candles_proximos = 0.3, diff_sombras=0.01, corpo_min = 3):

    # Buscando média de corpos
    corpo = abs(dados.close - dados.open)
    media_corpo = corpo.rolling(qtde_candles_proximos).min().shift(1)

    # Buscando candles com corpo pequeno
    sinal_corpo_pequeno = (corpo <= media_corpo * percentual_diff_candles_proximos) & (corpo >= corpo_min)

    # Filtrando por sombras quase iguais
    sombra_sup = dados.high - dados[['close', 'open']].max(axis=1)
    sombra_inf = dados[['close', 'open']].min(axis=1) - dados.low

    sinal_sombras = abs(sombra_sup - sombra_inf) <= diff_sombras

    sinal_candle = sinal_sombras & sinal_corpo_pequeno

    return sinal_candle

In [ ]:
padrao_candle = {}

for ticker, df_ohlc in dados_gerais.items():
    sinal_candle = encontrar_spinning_top(df_ohlc, diff_sombras = 0.1, corpo_min=0.05)
    if sinal_candle.sum() > 0:
        padrao_candle[ticker] = sinal_candle[sinal_candle]
padrao_candle

In [ ]:
plotar_candle(dados_gerais['PETR4'], data_candle = padrao_candle['PETR4'].index[0], qtde_ticks = 10)

### 2.4. Estrela

"As Estrelas são figuras singulares, formadas por um candle de pequenas proporções, com um corpo pequeno e sombras muito curtas (quando existem). Exceto pelo tamanho reduzido, não há nenhuma outra característica de formação específica para esse candle, podendo ser um Doji ou um Spinning."

"O que o torna uma Estrela é, na verdade, sua posição em relação aos candles anterior e posterior, que também fazem parte do padrão. Deve haver um gap de corpo (não de sombras) entre a Estrela e esses dois candles (anterior e posterior), e esses gaps devem ser em direções opostas, ou seja, <b> o corpo da Estrela não deve "tocar" o corpo do candle anterior e nem do candle posterior [...]"

"[...] Estrelas são válidas como sinal de reversão [...]. Quando aparece em uma tendência de baixa, é chamada Estrela da Manhã [...]; quando aparece em uma tendência de baixa, é chamada Estrela da Tarde [...]."

"A cor do segundo candle que representa a Estrela não é relevante para o padrão, contudo, para os candles anterior e posterior à Estrela, é necessário que a cor esteja alinhada com a tendência em que estão inseridos [...]."

"Um fator que agrega credibilidade ao padrão é o grau de penetração do terceiro candle (em sentido contrário) sobre o corpo do primeiro candle [...]"

In [ ]:
def encontrar_estrela(dados, qtde_candles_proximos = 5, percentual_diff_candles_proximos = 0.3):

    # Buscando candles com corpo pequeno
    candles_total = abs(dados.high - dados.low)
    media_candles_total = candles_total.rolling(qtde_candles_proximos).mean().shift(1)
    sinal_candle_pequeno = (candles_total <= media_candles_total * percentual_diff_candles_proximos)

    # Buscando candles com gap de corpo
    gap_baixa = (dados.open < dados[['close', 'open']].min(axis=1).shift(1)) & (dados.close < dados[['close', 'open']].min(axis=1).shift(1))
    gap_alta = (dados.open > dados[['close', 'open']].max(axis=1).shift(1)) & (dados.close > dados[['close', 'open']].max(axis=1).shift(1))


    # Identificando a tendência
    mm_50 = dados.close.rolling(50).mean()
    mm_9 = dados.close.ewm(span=9).mean()

    tendencia_alta = mm_9 > mm_50
    tendencia_baixa = mm_9 < mm_50

    candle_alta = dados.close > dados.open 
    candle_baixa = dados.close < dados.open

    # Buscando candles cujo candle anterior tenha gap para seu anterior e para o atual
    sinal_gaps_estrela_manha = (gap_baixa.shift(1)) & (gap_alta) & (tendencia_baixa.shift(2) & candle_baixa.shift(2))
    sinal_gaps_estrela_tarde = (gap_alta.shift(1)) & (gap_baixa) * (tendencia_alta.shift(2) & candle_alta.shift(2))

    sinal_candle = (sinal_gaps_estrela_tarde | sinal_gaps_estrela_manha) & (sinal_candle_pequeno.shift(1))

    return sinal_candle

In [ ]:
padrao_candle = {}

for ticker, df_ohlc in dados_gerais.items():
    sinal_candle = encontrar_estrela(df_ohlc, percentual_diff_candles_proximos=0.4, qtde_candles_proximos=3)
    if sinal_candle.sum() > 0:
        padrao_candle[ticker] = sinal_candle[sinal_candle]
padrao_candle

In [ ]:
plotar_candle(dados_gerais['WIN$'], data_candle = padrao_candle['WIN$'].index[0], qtde_ticks = 50)

## 3. Padrões Altistas

"Esses padrões são encontrados após tendências de baixa nos preços: localizam-se principalmente nos fundos dos gráficos e sinalizam ao investidor a possibilidade de reversão dessa tendência, indicando pontos de entrada no papel"

### 3.1. Martelo

"O Martelo é uma figura formada por um único candle. [...] se caracteriza por um candle de corpo pequeno com uma longa sombra inferior. A sombra inferior deve ter, no mínimo, duas vezes e meia o tamanho do corpo para ser um Martelo."

"Quanto maior a sombra inferior, mais significativo é o sinal"

"Não deve ter sombra superior ou, se tiver, deve ser bem pequena, insignificante."

"Para que a figura seja um Martelo, ela deve surgir depois de uma tendência mais prolongada de baixa ou após uma queda rápida dos preços em apenas alguns pregões. Se essa mesma figura aparecer depois de um movimento de alta ou durante uma acumulação, não é Martelo [...]"

"Se o tempo de vida de uma tendência for muito curta, o sinal tende a ser menos efetivo, podendo se configurar mais numa correção do que numa reversão"

"A cor do Martelo não é um fator importante, todavia sempre daremos certa preferência para a figura branca, já que representa uma alta no dia."

"Deve haver um acréscimo de volume no dia em que aparece o Martelo, o que fortalece o padrão, dando-lhe mais confiabilidade. Se tal acréscimo de volume não ocorrer no dia, porém no pregão seguinte, e esse dia for um candle de alta, o padrão estará validado."

"[...] o Martelo é um ótimo formador de fundos e, geralmente, marca um novo suporte de preços para o papel. Observe se o Martelo ocorre junto a um suporte (um fundo) já formado anteriormente ou próximo a uma Banda de Bollinger. Esses fatores aumentam muito sua confiabilidade"

In [ ]:
def encontrar_martelo(dados, relacao_sombra_corpo=2.5, relacao_sombra_sup_sombra_inf = 0.1, qtde_candle_tendencia = 5):

    # Encontrando candles com sombra inferior muito grande
    candle_martelo = dados[['close', 'open']].min(axis=1) - dados.low >= relacao_sombra_corpo * abs(dados.close - dados.open)

    # Filtrando por candles sem sombra superior (ou sombra insignificante)
    sem_sombra_sup = dados.high - dados[['close', 'open']].max(axis=1) <= relacao_sombra_sup_sombra_inf * (dados[['close', 'open']].min(axis=1) - dados.low)

    candle_martelo = candle_martelo & sem_sombra_sup

    
    # Identificando a tendência
    mm_50 = dados.close.rolling(50).mean()
    mm_9 = dados.close.ewm(span=9).mean()

    tendencia_baixa = (mm_9 < mm_50).rolling(qtde_candle_tendencia).mean() == 1


    # Confirmando com volume
    aumento_volume = dados.real_volume >= dados.real_volume.rolling(qtde_candle_tendencia).max()


    sinal_candle = candle_martelo & tendencia_baixa & aumento_volume

    return sinal_candle

In [ ]:
padrao_candle = {}

for ticker, df_ohlc in dados_gerais.items():
    sinal_candle = encontrar_martelo(df_ohlc)
    if sinal_candle.sum() > 0:
        padrao_candle[ticker] = sinal_candle[sinal_candle]


padrao_candle

In [ ]:
plotar_candle(dados_gerais['VALE3'], data_candle = padrao_candle['VALE3'].index[0], qtde_ticks = 60)

### 3.2. Martelo Invertido

"Martelo invertido se caracteriza por um candle de corpo pequeno com uma longa sombra superior, semelhante ao Martelo, mas de cabeça para baixo."

"A sombra superior <b>deve ser, no mínimo, duas vezes e meia o tamanho do corpo para ser um Martelo Invertido</b>. Não deve ter sombra inferior ou, se tiver, deve ser bem pequena, insignificante."

"[...] e,a deve surgir após uma tendência mais prolongada de baixa ou depois de uma queda rápida dos preços em apenas alguns pregões. Se essa mesma figura aparecer depois de um movimento de alta ou durante uma acumulação, não é um Martelo Invertido [...]"

"Deve haver acréscimo de volume no dia em que aparece o Martelo Invertido, o que fortalece o padrão, dando-lhe mais confiabilidade."

"Quanto mais a sombra superior do Martelo Invertido penetrar no corpo do candle do pregão anterior, melhor é o seu posicionamento [...]. Melhor ainda se o fechamento de um Martelo Invertido ficar dentro do corpo do candle de baixa do pregão anterior."

"O Martelo Invertido é um padrão de candle de confiabilidade média, sendo necessário esperar um ou dois dias para confirmar sua eficácia. Um bom sinal de que realmente está nascendo uma tendência de alta é o fato de o dia seguinte ser um candle de alta, abrindo ou pelo menos fechando acima do corpo do Martelo Invertido."

In [ ]:
def encontrar_martelo_invertido(dados, relacao_sombra_corpo=2.5, relacao_sombra_inf_sombra_sup = 0.1, 
                                qtde_candle_tendencia = 5, perc_confirmacao_tendencia=0.8):

    # Encontrando candles com sombra superior muito grande
    candle_martelo_invertido = dados.high - dados[['close', 'open']].max(axis=1) >= relacao_sombra_corpo * abs(dados.close - dados.open)

    # Filtrando por candles sem sombra inferior (ou sombra insignificante)
    sem_sombra_inf = dados[['close', 'open']].min(axis=1) - dados.low <= relacao_sombra_inf_sombra_sup * (dados.high - dados[['close', 'open']].max(axis=1))

    candle_martelo_invertido = candle_martelo_invertido & sem_sombra_inf

    
    # Identificando a tendência
    mm_50 = dados.close.rolling(50).mean()
    mm_9 = dados.close.ewm(span=9).mean()

    tendencia_baixa = (mm_9 < mm_50).rolling(qtde_candle_tendencia).mean() >= perc_confirmacao_tendencia


    # Confirmando com volume
    aumento_volume = dados.real_volume >= dados.real_volume.rolling(qtde_candle_tendencia).max()


    sinal_candle = candle_martelo_invertido & tendencia_baixa & aumento_volume


    # Aguardando dois dias para confirmação
    confirmacao_candle_posterior = (dados.close > dados[['close', 'open']].max(axis=1).shift(1)) | (dados.open > dados[['close', 'open']].max(axis=1).shift(1))
    sinal_candle_confirmado = (sinal_candle.shift(1)) & (confirmacao_candle_posterior)


    return sinal_candle_confirmado


padrao_candle = {}

for ticker, df_ohlc in dados_gerais.items():
    sinal_candle = encontrar_martelo_invertido(df_ohlc)
    if sinal_candle.sum() > 0:
        padrao_candle[ticker] = sinal_candle[sinal_candle]


padrao_candle

In [ ]:
plotar_candle(dados_gerais['WIN$'], data_candle = padrao_candle['WIN$'].index[0], qtde_ticks = 100)

### 3.3. Engolfo de Alta

"O Engolfo de Alta é uma figura formada por dois candles consecutivos. O primeiro possui um pequeno corpo negro, de baixa, enquanto o segundo possui um corpo bem maior e branco, de alta."

"O preço de abertura do segundo candle fica bem abaixo do preço de fechamento do primeiro candle, e o preço de fechamento do segundo candle fica bem acima do preço de abertura do primeiro candle, de forma que o segundo candle engolfa o primeiro num movimento contrário."

"O Engolfo de Alta deve aparecer necessariamente depois de uma tendência de baixa ou após uma queda rápida de preços [...]"

"Os dois candles que compõem o padrão podem ter sombras, mas é preferível que as sombras do primeiro candle - o preto - também fiquem engolfadas pelo corpo do segundo candle. Quanto maior a diferença de tamanho entre os dois candles, melhor."

"Deve-se observar o aumento considerável do volume no segundo dia do padrão, o que lhe confere força."

"[...] a proximidade de regiões de suporte junto ao "engolfo" aumenta a probabilidade da reversão."

In [ ]:
def encontrar_engolfo_de_alta(dados, relacao_entre_corpos=2.5, perc_bem_distante = 0.1,
                                qtde_candle_tendencia = 5, perc_confirmacao_tendencia=0.8):

    # Encontrando candles que tenham corpo grande em relação ao anterior
    corpo = abs(dados.close - dados.open)  
    candle_corpo_grande = corpo >= relacao_entre_corpos * corpo.shift(1)

    # Filtrando por candles cujo corpo engolfa o primeiro (com direções opostas - preto x branco ) com distancia min de "perc_bem_distante" %
    candle_engolfante = (dados.open < (1-perc_bem_distante) * dados.close.shift(1)) & (dados.close > (1+perc_bem_distante) * dados.open.shift(1))

    # Primeiro candle (ou seja, candle anterior) deve ser de baixa e o segundo de alta
    candle_de_baixa = dados.close < dados.open
    candle_de_alta = dados.close > dados.open 
    
    
    # Identificando a tendência de alta
    mm_50 = dados.close.rolling(50).mean()
    mm_9 = dados.close.ewm(span=9).mean()
    tendencia_baixa = (mm_9 < mm_50).rolling(qtde_candle_tendencia).mean() >= perc_confirmacao_tendencia


    # Confirmando com volume
    aumento_volume = dados.real_volume >= dados.real_volume.rolling(qtde_candle_tendencia).max().shift(1)

    sinal_padrao = candle_corpo_grande & candle_engolfante & candle_de_baixa.shift(1) & candle_de_alta & tendencia_baixa & aumento_volume

    return sinal_padrao


padrao_candle = {}

for ticker, df_ohlc in dados_gerais.items():
    sinal_candle = encontrar_engolfo_de_alta(df_ohlc, relacao_entre_corpos=3, perc_bem_distante=0, perc_confirmacao_tendencia=0.5, qtde_candle_tendencia=1)
    if sinal_candle.sum() > 0:
        padrao_candle[ticker] = sinal_candle[sinal_candle]


padrao_candle

In [ ]:
plotar_candle(dados_gerais['WDO$'], data_candle = padrao_candle['WDO$'].index[2], qtde_ticks = 40)

### 3.4. Harami de Fundo (<i>Bullish Harami</i>)

"É formado por dois candles consecutivos e deve sempre aparecer depois de uma tendência prolongada de baixa ou uma queda rápida de preços. <b>O primeiro dia é um longo candle de baixa (preto), e o segundo dia é um pequeno candle de alta (branco) com pouca sombra e, preferivelmente, com as sombras dentro do corpo do candle anterior</b> [...]"

"É importante que o corpo do segundo candle seja bem menor que o primeiro, algo em torno de um terço do tamanho." 

"Quando o candle do segundo dia for um Doji [...] teremos um Harami Cruz de Fundo."

"[...] não considero o padrão como um Harami se os dois candles não forem de cores diferentes."

"Esse padrão demonstra mais a fraqueza da tendência de baixa do que a força da tendência de alta, sendo menos significativo do que um Martelo ou um Engolfo de Alta, por exemplo."

In [ ]:
def encontrar_harami_fundo(dados, relacao_entre_corpos=3, sombra_dentro_corpo = True, 
                                qtde_candle_tendencia = 5, perc_confirmacao_tendencia=0.8):

    # Encontrando candles que tenham corpo pequeno em relação ao posterior
    corpo = abs(dados.close - dados.open)  

    candle_corpo_grande = corpo.shift(1) >= relacao_entre_corpos * corpo

    # Variável para ser conservador
    if sombra_dentro_corpo:
        # Sendo conservador na escolha do padrão e usando todo o candle dentro do corpo
        candle_dentro_anterior = (dados.low >= dados[['open', 'close']].min(axis=1).shift(1)) & (dados.high <= dados[['open', 'close']].max(axis=1).shift(1))

    else:
        candle_dentro_anterior = (dados[['open', 'close']].max(axis=1) <= dados[['open', 'close']].max(axis=1).shift(1)) \
                                & (dados[['open', 'close']].min(axis=1) >= dados[['open', 'close']].min(axis=1).shift(1))

    # Filtrando por candles que sejam de direções distintas (primeiro deve ser de baixa e o segundo de alta)
    dir_candle = (dados.close > dados.open) & (dados.close.shift(1) < dados.open.shift(1))  # Usando sempre o segundo candle como confirmação

    
    # Identificando a tendência de alta
    mm_50 = dados.close.rolling(50).mean()
    mm_9 = dados.close.ewm(span=9).mean()
    tendencia_baixa = (mm_9 < mm_50).rolling(qtde_candle_tendencia).mean() >= perc_confirmacao_tendencia


    sinal_padrao = candle_corpo_grande & candle_dentro_anterior & dir_candle & tendencia_baixa

    return sinal_padrao


padrao_candle = {}

for ticker, df_ohlc in dados_gerais.items():
    sinal_candle = encontrar_harami_fundo(df_ohlc, relacao_entre_corpos=2, perc_confirmacao_tendencia=0.8, qtde_candle_tendencia=1)
    if sinal_candle.sum() > 0:
        padrao_candle[ticker] = sinal_candle[sinal_candle]


padrao_candle

In [ ]:
plotar_candle(dados_gerais['WIN$'], data_candle = padrao_candle['WIN$'].index[2], qtde_ticks = 40)

Pelo visto, esse padrão realmente só representa uma fraqueza, mas não um ponto de virada propriamente dito. 

### 3.5. Linha de Perfuração (<i>Piercing Line Pattern</i>)

"É um importante padrão altista, formado por dois candles."

"[...] deve aparecer depois de uma tendência de baixa ou movimento rápido de queda."

"O primeiro candle é um corpo razoavelmente longo de baixo (preto); o segundo é de alta (branco), com o preço de abertura abaixo da mínima do dia anterior e preço de fechamento que avança sobre o corpo do candle anterior, atingindo mais de 50% de cobertura."

"A condição de avanço do segundo dia sobre o primeiro é de vital importância para o padrão, pois, sem cobrir no mínimo 50% do corpo de baixa do dia anterior, o padrão não é uma Linha de Perfuração [...]"

"Quanto mais baixo o preço de abertura do segundo dia em relação ao anterior, melhor."

"Quanto mais o segundo candle avançar sobre o corpo do primeiro dia, mais forte será seu poder de reversão."

In [ ]:
def encontrar_linha_perfuracao(dados, tamanho_gap = 0.01, percentual_avanco = 0.5, 
                                qtde_candle_tendencia = 5, perc_confirmacao_tendencia=0.8):

    # Primeira condição: 1º candle de baixo e 2º candle de alta
    cond_1 = (dados.open.shift(1) > dados.close.shift(1)) & (dados.close > dados.open)

    # Segunda condição: preço de abertura do 2º candle abaixo da mínima do candle anterior
    cond_2 = dados.open < dados.low.shift(1)

    # Terceira condição: preço de fechamento do 2º candle avança 50% do candle anterior 
    cond_3 = dados.close >= dados[['close', 'open']].mean(axis=1).shift(1)

    # Quarta condição: deve aparecer depois de uma tendência de baixa ou movimento rápido de queda
    mm_50 = dados.close.rolling(50).mean()
    mm_9 = dados.close.ewm(span=9).mean()
    cond_4 = (mm_9 < mm_50).rolling(qtde_candle_tendencia).mean() >= perc_confirmacao_tendencia

    # Filtro 1: diferença de preços entre a abertura do 2º candle e a mínima do candle anterior (tamanho do gap)
    filtro_1 = dados.low.shift(1) - dados.open >= tamanho_gap * abs(dados.close.shift(1) - dados.open.shift(1))

    # Filtro 2: valor do avanço do 2º candle sobre o primeiro
    filtro_2 = dados.close - dados[['close', 'open']].min(axis=1).shift(1) >= abs(dados.close - dados.open).shift(1) * percentual_avanco



    sinal_padrao = cond_1 & cond_2 & cond_3 & cond_4 & filtro_1 & filtro_2

    return sinal_padrao


padrao_candle = {}

for ticker, df_ohlc in dados_gerais.items():
    sinal_candle = encontrar_linha_perfuracao(df_ohlc, tamanho_gap=1)
    if sinal_candle.sum() > 0:
        padrao_candle[ticker] = sinal_candle[sinal_candle]


padrao_candle

In [ ]:
plotar_candle(dados_gerais['VALE3'], data_candle = padrao_candle['VALE3'].index[0], qtde_ticks = 40)

### 3.6. Pinça (ou Alicate) de Fundo (<i>Tweezer Bottom</i>)

"Esse padrão é formado por dois candles."

"Ele deve aparecer após um movimento de baixa."

"O primeiro candle possui um corpo mais longo e geralmente acompanha a tendência (corpo negro); o segundo candle pode ter um corpo menor, e sua mínima é igual (ou muito próxima) à mínima do dia anterior; <b>ou seja, eles estão 'alinhados pela mínima'</b>"

"[...] a cor do segundo candle <b>não</b> é tão importante, nem o seu tamanho."

"O ponto forte desse padrão é ter as mínimas alinhadas (ou quase alinhadas), pois esse fato atesta que ele marca um suporte [...]"

"Não é raro encontrarmos esse padrão acompanhado de um terceiro candle 'batendo' na mesma mínima, o que fortalece ainda mais o suporte gerado."

"Se ocorrer uma Pinça acompanhada de um terceiro candel com a mesma mínima, observar dois fatores que podem fortalecer a possibilidade de um futuro movimento de alta:

- Se o terceiro candle for negro, repare se seus preço de abertura é superior ao topo do corpo do segundo candle. Quanto mais ultrapassar esse topo, melhor, pois significa que o mercado abriu em 'alto-astral'
- Se o terceiro candle for branco, verifique se o preço de fechamento é superior ao topo do corpo do segundo candle. Esse é um sinal ainda melhor, porque, além de 'bater' na mesma mínima, os preços ainda subiram bastante."

"[...] quanto menor for a periodicidade do gráfico, menor também será sua confiabilidade."

In [ ]:
def encontrar_pinca_fundo(dados, max_diferenca_minimos = 0.001, relacao_min_corpo_total_1_candle = 0.5, 
                          relacao_max_corpo_total_2_candle = 0.25, qtde_candle_tendencia = 5,
                        perc_confirmacao_tendencia = 0.8, buscar_reforcado=False, reforco_superior_topo_anterior = 0.1):

    # Primeira condição: 2 candles seguidos de mínimas parecidas
    cond_1 = abs(dados.low - dados.low.shift(1)) <= max_diferenca_minimos * dados.low.shift(1)  # optou-se por usar percentual para ser mais genérico

    # Segunda condição: 1º candle ser negro com corpo longo
    cond_2 = (dados.open.shift(1) - dados.close.shift(1)) >= relacao_min_corpo_total_1_candle * (dados.high.shift(1) - dados.low.shift(1))

    # Terceira condição: 2º candle  com corpo pequeno (independentemente da cor)
    cond_3 = abs(dados.close - dados.open) <= relacao_max_corpo_total_2_candle * (dados.high - dados.low)

    # Quarta condição: deve aparecer depois de uma tendência de baixa ou movimento rápido de queda
    mm_50 = dados.close.rolling(50).mean()
    mm_9 = dados.close.ewm(span=9).mean()
    cond_4 = (mm_9 < mm_50).rolling(qtde_candle_tendencia).mean() >= perc_confirmacao_tendencia

    # Filtro 1: ser sequencia de 3 candles de mínima 
    if buscar_reforcado:
        filtro_1 = abs(dados.low.shift(1) - dados.low.shift(2)) <= max_diferenca_minimos * dados.low.shift(2)  # optou-se por usar percentual para ser mais genérico
    
        # Filtro 2: além do filtro 1, deter aos filtros de fortalecimento do terceiro candle
        # Utilizou-se um percentual como filtro, a fim de que possa filtrar pela "potência" do sinal (maior distancia ao topo anterior) 
        filtro_2 = dados[['open', 'close']].max(axis=1) - dados[['open', 'close']].max(axis=1).shift(1) >= reforco_superior_topo_anterior * dados[['open', 'close']].max(axis=1).shift(1)

   
        sinal_padrao = cond_1 & cond_2 & cond_3 & cond_4 & filtro_1 & filtro_2

    else:
        sinal_padrao = cond_1 & cond_2 & cond_3 & cond_4

    return sinal_padrao


padrao_candle = {}

for ticker, df_ohlc in dados_gerais.items():
    sinal_candle = encontrar_pinca_fundo(df_ohlc, max_diferenca_minimos=0.006, 
                                         relacao_max_corpo_total_2_candle=0.4, 
                                         buscar_reforcado=False,
                                         reforco_superior_topo_anterior=0)
    if sinal_candle.sum() > 0:
        padrao_candle[ticker] = sinal_candle[sinal_candle]


padrao_candle

In [ ]:
plotar_candle(dados_gerais['WDO$'], data_candle = padrao_candle['WDO$'].index[0], qtde_ticks = 100)

### 3.7. Chute de Alta (<i>Bullish Kicking</i>)

"Esse padrão é raro e representa uma mudança drástica na postura do mercado em relação ao papel."

"Aparece durante um movimento de baixa e é formado por dois candles longos. O primeiro é um candle negro, de baixa, sem sombras (vamos admitir pequenas sombras se forem insignificantes). O segundo é um candle de corpo branco, de alta, sem sombras (também se admitem pequenas sombras, se insignificantes), cujo preço de abertura esteja acima do preço de abertura do candle anterior."

"Se os candles tiverem sombras, a sombra superior do primeiro candle (negro) não deve tocar a sombra inferior do segundo candle(branco), configurando o que chamamos de <b>gap completo</b> ou <b>window</b>."

"[...] a força dessa reversão será proporcional ao aumento de volume observado no segundo dia do padrão. Quanto maior o volume observado, maior a euforia em que entrou o mercado para o papel."

"Pelo fato de o papel ter percorrido um longo caminho entre o fechamento do primeiro e o fechamento do segundo dia, o mercado ficará sujeito a correções, dependendo do grau de otimismo causado pela expectativa criada."


In [50]:
def encontrar_chute_alta(dados, max_relacao_sombras_corpo = 0.01, qtde_candle_tendencia = 5,
                        perc_confirmacao_tendencia = 0.8, filtro_volume = True, periodo_historico_volume = 10, prop_min_entre_volumes = 1.1):

    corpo_candle = abs(dados.close - dados.open)
    total_candle = dados.high - dados.low 
    sombra_sup = dados.high - dados[['close', 'open']].max(axis=1)
    sombra_inf = dados[['close', 'open']].min(axis=1) - dados.low
    
    # Primeira condição: 1º candle negro sem sombra significativa
    cond_1 = (dados.close.shift(1) < dados.open.shift(1)) & (sombra_sup.shift(1) < corpo_candle.shift(1) * max_relacao_sombras_corpo) & (sombra_inf.shift(1) < corpo_candle.shift(1) * max_relacao_sombras_corpo)

    # Segunda condição: 2º candle ser branco sem sombra significativa
    cond_2 = (dados.close > dados.open) & (sombra_sup < corpo_candle * max_relacao_sombras_corpo) & (sombra_inf < corpo_candle * max_relacao_sombras_corpo)

    # Terceira condição: 2º candle com com mínimo sem tocar máxima do dia anterior (Gap Completo / Window)
    cond_3 = dados.low > dados.high.shift(1)

    # Quarta condição: deve aparecer depois de uma tendência de baixa ou movimento rápido de queda
    mm_50 = dados.close.rolling(50).mean()
    mm_9 = dados.close.ewm(span=9).mean()
    cond_4 = (mm_9 < mm_50).rolling(qtde_candle_tendencia).mean() >= perc_confirmacao_tendencia


    sinal_padrao = cond_1 & cond_2 & cond_3 & cond_4

    # Filtro 1: se volume no 2º candle for maior que média histórica e maior que 1º candle 
    if filtro_volume:
        filtro_1 = (dados.real_volume > dados.real_volume.rolling(periodo_historico_volume).mean()) & (dados.real_volume > dados.real_volume.shift() * prop_min_entre_volumes)
        sinal_padrao = sinal_padrao & filtro_1
    return sinal_padrao


padrao_candle = {}

for ticker, df_ohlc in dados_gerais.items():
    sinal_candle = encontrar_chute_alta(df_ohlc, 
                                        max_relacao_sombras_corpo = 0.5, 
                                        qtde_candle_tendencia = 5,
                                        perc_confirmacao_tendencia = 0.8, 
                                        filtro_volume = False, 
                                        periodo_historico_volume = 10, 
                                        prop_min_entre_volumes = 1.1)
    if sinal_candle.sum() > 0:
        padrao_candle[ticker] = sinal_candle[sinal_candle]


padrao_candle

{'VALE3': time
 2022-04-27    True
 2024-04-08    True
 dtype: bool}

In [ ]:
plotar_candle(dados_gerais['VALE3'], data_candle = padrao_candle['VALE3'].index[0], qtde_ticks = 30)

In [ ]:
plotar_candle(dados_gerais['VALE3'], data_candle = padrao_candle['VALE3'].index[0], qtde_ticks = 30)